In [1]:
!pip install --upgrade tensorflow
!pip install tensorflow - gpu

     |████████████████████████████████| 497.6 MB 20 kB/s s eta 0:00:01|████▎                           | 65.8 MB 59.6 MB/s eta 0:00:08 |████▊                           | 73.5 MB 59.6 MB/s eta 0:00:08                    | 77.1 MB 59.6 MB/s eta 0:00:08         | 86.2 MB 59.6 MB/s eta 0:00:07MB 59.6 MB/s eta 0:00:07��███▏                      | 142.6 MB 62.6 MB/s eta 0:00:06�█████████▍                 | 224.6 MB 29.2 MB/s eta 0:00:10�█████████▉                 | 229.9 MB 29.2 MB/s eta 0:00:10/s eta 0:00:11��██████████████              | 278.9 MB 23.6 MB/s eta 0:00:10��██████████████▏             | 281.7 MB 23.6 MB/s eta 0:00:10MB/s eta 0:00:10MB/s eta 0:00:09��█▎            | 299.9 MB 23.4 MB/s eta 0:00:09��█▌            | 302.6 MB 23.4 MB/s eta 0:00:09 23.4 MB/s eta 0:00:09��██            | 309.3 MB 23.4 MB/s eta 0:00:09:00:03        | 344.7 MB 64.1 MB/s eta 0:00:03        | 352.5 MB 64.1 MB/s eta 0:00:03██████████████      | 403.5 MB 71.4 MB/s eta 0:00:02��██▎     | 408.4 MB 71.4 MB/s e

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

2022-04-16 11:53:35.089304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow


In [3]:
!pip install tensorflow_datasets

     |████████████████████████████████| 4.2 MB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 13.5 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=82a4dd5eaf4897e36750d395fe98f9767d9d647cf7f5975e947793165e206371
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/e1/e8/83/ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise


In [4]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
x_train, y_train = x_train_full[:50000], y_train_full[:50000]
x_valid, y_valid = x_train_full[50000:], y_train_full[50000:]

In [6]:
x_train.shape

(50000, 28, 28)

In [7]:
x_mean = x_train.mean(axis = 0, keepdims = True)
x_std = x_train.std(axis = 0, keepdims = True) + 1e-7
x_train = (x_train-x_mean)/x_std
x_valid = (x_valid - x_mean)/x_std
x_test = (x_test - x_mean)/x_std

In [8]:
from functools import partial
DefaultConv2D = partial(keras.layers.Conv2D,
                       kernel_size = 3, activation = 'relu', padding= 'SAME')

model = keras.models.Sequential([
    DefaultConv2D(filters = 64, kernel_size = 7, input_shape = [28,28,1]),
    keras.layers.MaxPooling2D(pool_size = 2),
    DefaultConv2D(filters = 128),
    DefaultConv2D(filters = 128),
    keras.layers.MaxPooling2D(pool_size = 2),
    DefaultConv2D(filters = 128),
    DefaultConv2D(filters = 128),
    keras.layers.MaxPooling2D(pool_size = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 128, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 64, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 10, activation = 'softmax')
])

2022-04-16 11:53:46.375727: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-04-16 11:53:46.375780: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        3200      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         1

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'nadam', metrics = 'accuracy')
history = model.fit(x_train, y_train, epochs = 10, validation_data = (x_valid, y_valid))
score = model.evaluate(x_test, y_test)

In [ ]:
y_pred = model.predict(x_test[:10])
